In [8]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [7]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [22]:
## District Summary

# Calculate the total number of schools
total_schools = school_data['School ID'].count()

# Calculate the total number of students
total_students = student_data['Student ID'].count()

# Calculate the total budget
total_budget = school_data['budget'].sum()

# Calculate the average math score
average_math_score = student_data['math_score'].mean()
 
# Calculate the average reading score
average_reading_score = student_data['reading_score'].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (average_math_score + average_reading_score) / 2

# Calculate the percentage of students with a passing math score (70 or greater)
gender_summary['Percentage of Players'] = gender_summary['Gender'] / gender_summary['Gender'].sum()

# Calculate the percentage of students with a passing reading score (70 or greater)

# Create a dataframe to hold the above results
school_district_summary = pd.DataFrame([(total_schools, total_students, total_budget, average_math_score, average_reading_score, overall_passing_rate)], columns=['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score','Overall Passing Rate'])

school_district_summary
# Optional: give the displayed data cleaner formatting

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,80.431606
